In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl
import anndata as ann
import random, os
from scipy.stats import pearsonr as pr
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score as f1
from sklearn.metrics import precision_recall_curve as prc
from sklearn.metrics import silhouette_score as sil
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, average_precision_score
from sklearn.metrics import silhouette_score
from torch_geometric.nn import TransformerConv
from torch_geometric.data import Data
import gc, psutil

In [2]:
import torch
import torch_geometric
from torch_geometric.nn import GraphConv, GCNConv
from sklearn.model_selection import StratifiedKFold
import scanpy as sc
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from torch_geometric.utils import from_scipy_sparse_matrix
from sklearn.preprocessing import LabelEncoder
from torch_geometric.loader import NeighborLoader


In [3]:
sc.set_figure_params(dpi=200)

In [4]:
import torch
from torch_geometric.data import DataLoader
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader
from tqdm import tqdm  # For progress bar
from sklearn.model_selection import KFold

# Set random seeds for reproducibility
import random
import numpy as np
import torch

def set_random_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # if using CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seeds(42)
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from torch.nn import Linear, Dropout
from torch_geometric.nn import TransformerConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, accuracy_score


In [6]:
from torch_geometric.data import Data
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [7]:
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    average_precision_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from torch_geometric.loader import RandomNodeLoader
from torch_geometric.utils import subgraph

In [8]:
def memory_usgae():
    gc.collect()
    torch.cuda.empty_cache()
    process = psutil.Process(os.getpid())
    memory_gb = process.memory_info().rss / 1024**3  # in GB

    print(f"Current memory usage: {memory_gb:.2f} GB")

# Load data

In [9]:
# load the data
ad = sc.read_h5ad('./Data/Cancer_cell_data/All_integrated.hallmark.harmony.h5ad')
ad

AnnData object with n_obs × n_vars = 725643 × 2132
    obs: 'Project_ID', 'Primary_or_Metastatic', 'Final_cancer_type', 'Final_histological_subtype', 'Final_molecular_subtype', 'Final_tissue', 'Final_sample_id', 'Final_patient_age', 'Final_patient_stage', 'Final_patient_treatment', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'Final_histological_subtype_backup', 'Final_patient_age_backup', 'Final_tissue_backup', 'Final_patient_treatment_backup', 'Final_patient_stage_backup', 'Classifier_label'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'Final_cancer_type_colors', 'Final_histological_subtype_colors', 'Final_molecular_subtype_colors', 'Final_patient_stage_colors', 'Final_patient_treatment_colors', 'Final_tissue_colors', 'Primary_or_Metastatic_colors', 'Project_ID_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_pca_harmony_Project_ID', 'X_pca_harmony_project_id', 'X_uma

Label unification:

Primary: Non-metastatic Local

Local: Metastatic Local

Distant: Metastatic distant

In [10]:
# Map cancer type → its "primary tissue"
primary_tissue_map = {
    "Breast Cancer": "Breast",
    "Lung Cancer": "Lung",
    "Ovarian Cancer": "Ovary",
    "Colorectal Cancer": "Colon"
}

# Initialize as primary
ad.obs["source"] = "Primary"


# Loop through rows
for idx, row in ad.obs.iterrows():
    cancer_type = row["Final_cancer_type"]
    tissue = row["Final_tissue"]

    primary_site = primary_tissue_map.get(cancer_type, None)

    if row["Primary_or_Metastatic"] == "Primary":
        ad.obs.at[idx, "source"] = "Primary"

    else:  # Metastatic
        if tissue == primary_site:
            ad.obs.at[idx, "source"] = "local"
        else:
            ad.obs.at[idx, "source"] = "distant"

# Summary:
print(ad.obs["source"].value_counts())


source
Primary    328282
distant    249125
local      148236
Name: count, dtype: int64


In [11]:
memory_usgae()

Current memory usage: 21.07 GB


# Model

In [12]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Dropout
from torch_geometric.nn import TransformerConv

class scMeta(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, heads=4, dropout=0.3):
        super(scMeta, self).__init__()

        # Transformer layers
        self.conv1 = TransformerConv(in_channels=input_dim, out_channels=hidden_dim, heads=heads, dropout=dropout)
        self.conv2 = TransformerConv(in_channels=hidden_dim * heads, out_channels=hidden_dim, heads=1, dropout=dropout)

        # Classifier head
        self.classifier = torch.nn.Sequential(
            Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            Dropout(dropout),
            Linear(hidden_dim, num_classes)
        )

    def forward(self, x, edge_index, return_embedding=False):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        logits = self.classifier(x)

        if return_embedding:
            return logits, x  # logits, embeddings
        else:
            return logits


# Data preperation

In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from sklearn.model_selection import StratifiedKFold

# AnnData copy
ad_sub = ad.copy()

# Features
if not isinstance(ad_sub.X, np.ndarray):
    X = torch.tensor(ad_sub.X.toarray(), dtype=torch.float32)
else:
    X = torch.tensor(ad_sub.X, dtype=torch.float32)

# Graph edges
adj = ad_sub.obsp["connectivities"].tocoo()
edge_index = torch.tensor(np.vstack((adj.row, adj.col)), dtype=torch.long)

# Labels
primary_mask = (ad_sub.obs["Primary_or_Metastatic"] == "Primary").values
local_mask = (ad_sub.obs["Primary_or_Metastatic"] == "Metastatic") & (ad_sub.obs["source"] == "local").values
distant_mask = (ad_sub.obs["Primary_or_Metastatic"] == "Metastatic") & (ad_sub.obs["source"] == "distant").values

y_3class = np.full(ad_sub.n_obs, -1, dtype=int)
y_3class[primary_mask] = 0
y_3class[local_mask] = 1
y_3class[distant_mask] = 2
y_3class = torch.tensor(y_3class, dtype=torch.long)

# Binary label for Primary vs Local mets only (for filtered evaluation)
y_binary = np.full(ad_sub.n_obs, -1, dtype=int)
y_binary[primary_mask] = 0
y_binary[local_mask] = 1
y_binary = torch.tensor(y_binary, dtype=torch.long)
valid_mask = (y_3class == 0) | (y_3class == 1)
valid_idx = np.where(valid_mask)[0]

# Stratified 5-fold CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, val_idx in skf.split(valid_idx, y_3class[valid_idx]):
    folds.append({
        "train_idx": valid_idx[train_idx],
        "val_idx": valid_idx[val_idx]
    })

data = Data(
    x=X,
    edge_index=edge_index,
    y=y_3class
)
data.cv_folds = folds

print(f"Prepared {data.num_nodes} nodes")
print(f"Primary: {(y_3class==0).sum().item()} | Local: {(y_3class==1).sum().item()} | Distant: {(y_3class==2).sum().item()}")
print("Stratified 5-fold CV done.")


# Training

In [17]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

def evaluate(model, x_val, y_val, y_binary, device):
    model.eval()

    # Self-loop edges only
    N = x_val.size(0)
    edge_index_val = torch.arange(N, device=device).unsqueeze(0).repeat(2, 1)

    with torch.no_grad():
        logits, _ = model(x_val, edge_index=edge_index_val, return_embedding=True)

    y_true_3class = y_val.cpu().numpy()
    y_pred_3class = logits.argmax(dim=1).cpu().numpy()
    y_prob_3class = torch.softmax(logits, dim=1).cpu().numpy()
    
    y_binary = y_binary.to(device)
    y_val = y_val.to(device)


    # Filter for binary classification (primary vs local mets)
    binary_mask = (y_binary != -1) & (y_val != 2)  # exclude distant mets
    
    y_true_bin = y_binary[binary_mask].cpu().numpy()
    y_pred_bin = logits.argmax(dim=1)[binary_mask].cpu().numpy()
    y_prob_bin = torch.softmax(logits, dim=1)[binary_mask, :2].cpu().numpy()
    
    # Compute metrics
    try:
        auc = roc_auc_score(y_true_bin, y_prob_bin[:, 1], multi_class='ovr', average='weighted')
    except:
        auc = np.nan

    try:
        auprc = average_precision_score(y_true_bin, y_prob_bin[:, 1], average='weighted')
    except:
        auprc = np.nan

    acc = accuracy_score(y_true_bin, y_pred_bin)
    f1 = f1_score(y_true_bin, y_pred_bin, average='weighted')
    
    return acc, f1, auc, auprc, y_true_bin, y_pred_bin


In [15]:
import torch.optim as optim
from torch_geometric.loader import RandomNodeLoader
from torch_geometric.utils import subgraph
from tqdm import tqdm

def NT_Xent(embeddings, tau=0.5):
    device = embeddings.device
    z_i = F.normalize(embeddings, dim=1)
    z_j = F.normalize(embeddings[torch.randperm(z_i.size(0))], dim=1)

    logits = torch.mm(z_i, z_j.t()) / tau
    labels = torch.arange(z_i.size(0), device=device)
    loss = F.cross_entropy(logits, labels)
    return loss

def run_fold_cv(data, save_dir, epochs=100, seed=42, patience=10, gamma=0.1):
    os.makedirs(save_dir, exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for fold, fold_dict in enumerate(data.cv_folds):
        memory_usgae()

        print(f"\n🔁 Fold {fold + 1}")

        train_idx = fold_dict["train_idx"]
        val_idx = fold_dict["val_idx"]

        train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        train_mask[train_idx] = True

        # Build train graph
        edge_index_train, _ = subgraph(train_mask, data.edge_index, relabel_nodes=True)
        x_train = data.x[train_mask]
        y_train = data.y[train_mask]

        train_data = Data(
            x=x_train,
            edge_index=edge_index_train,
            y=y_train
        )

        x_val = data.x[val_idx].to(device)
        y_val = data.y[val_idx].to(device)
        
        y_binary_val = y_binary[val_idx]


        model = scMeta(
            input_dim=data.num_node_features,
            hidden_dim=128,
            num_classes=len(torch.unique(data.y))
        ).to(device)

        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        best_acc = 0
        best_auc = 0.0
        patience_counter = 0
        best_model_path = os.path.join(save_dir, f"fold{fold + 1}_best_model.pt")
        
        # double check class distribution in val set
        # no distant mets should be here, no label 2
        y_val_counts = torch.bincount(data.y[val_idx])
        for label, count in enumerate(y_val_counts):
            print(f"Val label {label}: {count.item()} samples")

        # Pre-training eval
        acc, f1, auc, auprc, _, _ = evaluate(
            model, x_val, y_val, y_binary_val, device
        )
        print(f"Pre-training stats: Acc={acc:.4f}, F1={f1:.4f}, AUROC={auc:.4f}, AUPRC={auprc:.4f}")

        for epoch in tqdm(range(epochs), desc=f"Fold {fold + 1}"):
            model.train()
            total_loss = 0

            loader = RandomNodeLoader(train_data, num_parts=100, shuffle=True)
            for batch in loader:
                batch = batch.to(device)
                logits, emb = model(batch.x, batch.edge_index, return_embedding=True)

                loss_ce = F.cross_entropy(logits, batch.y)
                loss_con = NT_Xent(emb, tau=0.5)
                loss = loss_ce + gamma * loss_con

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            # Evaluate
            acc, f1, auc, auprc, _, _ = evaluate(
                model, x_val, y_val, y_binary_val, device
            )

            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Acc={acc:.4f}, F1={f1:.4f}, AUROC={auc:.4f}, AUPRC={auprc:.4f}")

            if auc > best_auc:
                best_auc = auc
                torch.save(model.state_dict(), best_model_path)
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"⏹️ Early stopping at epoch {epoch + 1}")
                    break

        print(f"✅ Best Acc for Fold {fold + 1}: {best_auc:.4f}")

    print("🎉 Finished 5-fold CV with contrastive + CE loss.")


In [18]:
for gamma in [0.0, 0.1, 0.2, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0]:
    print('*'*10, gamma, '*'*10)
    save_dir = '/scratch/gilbreth/wang3712/Metastasis_single_cell/scMeta_5fol_CV_gamma'+str(gamma)+'/'
    

    run_fold_cv(
        data,
        save_dir=save_dir,
        epochs=100,
        patience=10,
        gamma=gamma
    )

********** 4.0 **********
Current memory usage: 19.55 GB

🔁 Fold 1
Val label 0: 65657 samples
Val label 1: 29647 samples
Pre-training stats: Acc=0.1553, F1=0.2390, AUROC=0.4168, AUPRC=0.2660


Fold 1:   1%|          | 1/100 [00:12<20:35, 12.48s/it]

Epoch 0: Acc=0.8940, F1=0.8912, AUROC=0.9435, AUPRC=0.9132


Fold 1:  11%|█         | 11/100 [02:14<18:06, 12.20s/it]

Epoch 10: Acc=0.9788, F1=0.9788, AUROC=0.9974, AUPRC=0.9951


...

Epoch 80: Acc=0.9860, F1=0.9860, AUROC=0.9989, AUPRC=0.9978


Fold 5:  91%|█████████ | 91/100 [18:32<01:49, 12.20s/it]

Epoch 90: Acc=0.9870, F1=0.9870, AUROC=0.9988, AUPRC=0.9977


Fold 5:  95%|█████████▌| 95/100 [19:33<01:01, 12.35s/it]

⏹️ Early stopping at epoch 96
✅ Best Acc for Fold 5: 0.9989
🎉 Finished 5-fold CV with contrastive + CE loss.


# Evaluate

This is what is reported in the paper.

In [21]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_results = dict()
for gamma in [0.0, 0.1, 0.2, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0]:
    print('*'*20, gamma, '*'*20)
    all_results[gamma] = dict()
    save_dir = '/scratch/gilbreth/wang3712/Metastasis_single_cell/scMeta_5fol_CV_gamma'+str(gamma)+'/'

    class_names = ["Non-metastatic local", "Metastatic local", "Metastatic distant"]

    all_y_true = []
    all_y_pred = []
    all_y_prob = []

    metrics_df = []

    for fold, fold_dict in enumerate(data.cv_folds):
        val_idx = fold_dict["val_idx"]
        x_val = data.x[val_idx].to(device)
        y_val = data.y[val_idx].to(device)
        y_binary_val = y_binary[val_idx]

        model = scMeta(
            input_dim=data.num_node_features,
            hidden_dim=128,
            num_classes=3
        ).to(device)

        model_path = os.path.join(save_dir, f"fold{fold + 1}_best_model.pt")
        if not os.path.exists(model_path):
            print(f"Missing model {model_path}, skipping")
            continue

        model.load_state_dict(torch.load(model_path))
        
        model.eval()

        acc, f1, auc, auprc, _, _ = evaluate(
            model, x_val, y_val, y_binary_val, device
        )
        
        
        metrics_df.append({
            "Fold": fold + 1,
            "ACC": acc,
            "F1": f1,
            "AUROC": auc,
            "AUPRC": auprc
        })

    # Save metrics
    df_metrics = pd.DataFrame(metrics_df).set_index("Fold")
    display(df_metrics)
    all_results[gamma] = df_metrics


******************** 0.0 ********************


,ACC,F1,AUROC,AUPRC
Fold,,,,
1,0.986601,0.986591,0.998690,0.997547
2,0.986034,0.986044,0.998678,0.997440
3,0.983873,0.983907,0.998555,0.997206
4,0.986580,0.986597,0.998862,0.997807
5,0.987125,0.987123,0.998819,0.997772


******************** 0.1 ********************


,ACC,F1,AUROC,AUPRC
Fold,,,,
1,0.984618,0.984638,0.998600,0.997316
2,0.985761,0.985761,0.998610,0.997328
3,0.986485,0.986460,0.998877,0.997780
4,0.986370,0.986373,0.998889,0.997802
5,0.985667,0.985670,0.998779,0.997578


******************** 0.2 ********************


...

,ACC,F1,AUROC,AUPRC
Fold,,,,
1,0.986622,0.986601,0.998868,0.997795
2,0.984670,0.984655,0.998490,0.997072
3,0.986139,0.986115,0.998782,0.997604
4,0.987314,0.987315,0.998978,0.998013
5,0.986559,0.986556,0.998949,0.997893


******************** 4.0 ********************


,ACC,F1,AUROC,AUPRC
Fold,,,,
1,0.985237,0.985187,0.998668,0.997442
2,0.986160,0.986155,0.998734,0.997506
3,0.987083,0.987071,0.998964,0.997944
4,0.986412,0.986401,0.998775,0.997648
5,0.986926,0.986940,0.998995,0.998029


******************** 5.0 ********************


,ACC,F1,AUROC,AUPRC
Fold,,,,
1,0.987314,0.987295,0.998910,0.997891
2,0.986223,0.986226,0.998844,0.997605
3,0.984932,0.984964,0.998751,0.997603
4,0.986024,0.986011,0.998794,0.997571
5,0.987083,0.987075,0.998929,0.997931


In [26]:
metrics = ['ACC', 'F1', 'AUROC', 'AUPRC']
for gamma in all_results.keys():
    print('*'*20, gamma, '*'*20)
    res = []
    for metric in metrics:
        res.append((all_results[gamma][metric].mean()))
    print(res)
    print(np.mean(res))

******************** 0.0 ********************
[0.9860425033680016, 0.9860522990925726, 0.998720882952699, 0.9975545547207133]
0.9920925600334967
******************** 0.1 ********************
[0.985780181391247, 0.9857805095883887, 0.9987509736326154, 0.9975608576843614]
0.9919681305741531
******************** 0.2 ********************
[0.9857507976483253, 0.9857518600148234, 0.9987505929291247, 0.997586331746572]
0.9919598955847114
******************** 0.5 ********************
[0.985872516503775, 0.9858725081326352, 0.9988271966451739, 0.9976950109390552]
0.9920668080551598
******************** 1.0 ********************
[0.9859774470428446, 0.9859787287881063, 0.9987566560083032, 0.9975822296332902]
0.9920737653681361
******************** 2.0 ********************
[0.9859480671533781, 0.9859595601579804, 0.9988280062331913, 0.9977256011344823]
0.992115308669758
******************** 3.0 ********************
[0.9862607526920291, 0.9862484382575186, 0.9988131943663445, 0.9976754114341884]
0.